In [212]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
%matplotlib inline


def connect_production_db():
    conn = psycopg2.connect("dbname='chef_buddy' user='chefbuddyadmin' host='cooper.cdhlqbkfdy0e.us-west-2.rds.amazonaws.com' password='chefbuddy123'")
    conn.set_session(autocommit=True)
    return conn

def connect_local_db():
    conn = psycopg2.connect("dbname='chef_buddy' user='zackjcooper' host='127.0.0.1' password='password'")
    conn.set_session(autocommit=True)
    return conn

connection = connect_production_db()
cur = connection.cursor()

def get_recipe_fc(recipe_id):
    df = pd.read_sql("SELECT * from chef_buddy_recipe where recipe_id='" + recipe_id + "'" , connection)
    return df

def get_user_fc(user_id):
    df = pd.read_sql("SELECT * from chef_buddy_userflavorcompound where user_id='" + str(user_id) + "'" , connection)
    return df
def get_ingredient_fc(ingredient):
    df = pd.read_sql("SELECT * from chef_buddy_ingredientflavorcompound where ingredient_id='" + ingredient + "'" , connection)
    return df.flavor_id

def get_unique_ingredient_list():
    df = pd.read_sql("SELECT DISTINCT ingredient_id from chef_buddy_ingredientflavorcompound;" , connection)
    return df.ingredient_id

def ingredient_lookup(flavor_id):
    cur.execute("select ARRAY (select ingredient_id from chef_buddy_ingredientflavorcompound where flavor_id = '" +str(flavor_id)+ "');")
    return cur.fetchall()

def users_ingredient_score(user_id):
    ufc_df = get_user_fc(user_id)
    ing_dict = {}
    
    for count in range(len(ufc_df)):
        ingredient_list = ingredient_lookup(ufc_df.flavor_id[count])
        
        for ingredient in ingredient_list[0][0]:
            if ingredient in ing_dict.keys():
                ing_dict[ingredient] += ufc_df.score[count]
            else:
                ing_dict[ingredient] = ufc_df.score[count]
    return OrderedDict(sorted(ing_dict.items(), key=lambda t: t[1]))

ing_fc_all = pd.read_sql("SELECT * from chef_buddy_ingredientflavorcompound", connection)

In [200]:
recipe_df = get_recipe_fc('Thick-and-Cakey-Chocolate-Chip-Cookies-677686')
ufc_df = get_user_fc(2)
ing_fc_df = get_ingredient_fc('butter')

In [201]:
user_ingr = users_ingredient_score(2)
user_ingr

OrderedDict([('bell pepper', -38), ('black pepper', -38), ('long pepper', -37), ('malagueta pepper', -36), ('green bell pepper', -36), ('capsicum annuum', -36), ('guinea pepper', -36), ('capsicum', -36), ('ethiopian pepper', -36), ('cayenne', -36), ('tabasco pepper', -36), ('thai pepper', -36), ('ashanti pepper', -36), ('california pepper', -36), ('pepper', -36), ('garlic', -21), ('skim milk', -17), ('filbert', -12), ('myrtle', -10), ('parsley', -10), ('dried parsley', -10), ('onion', -9), ('goat milk', -8), ('dairy', -8), ('sheep milk', -8), ('sour milk', -8), ('mint', -8), ('mace', -8), ('japanese mint', -7), ('caucas', -7), ('pimento', -7), ('roasted onion', -7), ('sage', -6), ('fennel', -6), ('eucalyptus oil', -6), ('nutmeg', -6), ('durian', -6), ('laurel', -6), ('cumin', -6), ('mint oil', -6), ('caucus', -6), ('nectarine', -6), ('eucalyptus', -5), ('currant', -5), ('naranjilla', -5), ('corn salad', -5), ('lemon balm', -5), ('nira', -5), ('anise', -5), ('camphor oil', -5), ('nobiru

In [214]:
ufc_df = set(ufc_df)
    
for ingredient in get_unique_ingredient_list():
    ingr_fc_list = set(get_ingredient_fc(ingredient))
    in_common = set.intersection(ufc_df, ingr_fc_list)
    if len(in_common) == len(ingr_fc_list):
        print(ingredient)